<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/107_%E5%91%BC%E3%81%B3%E5%87%BA%E3%81%97%E5%8F%AF%E8%83%BD%E5%9E%8B%E3%81%A8%E3%83%87%E3%82%B3%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

呼び出し可能型とデコレーター
============================

呼び出し可能型
--------------

`__call__()` メソッドが定義されている型を、**呼び出し可能型**と呼ぶ。

組み込み関数 `callable(object)` は、`object` 引数が呼び出し可能オブジェクトであれば `True` を、そうでなければ `False` を返す。

呼び出し可能オブジェクトは、`()` を付けて呼び出すことができる。`x(arg1, ...)` のようなオブジェクトの呼び出しは、実行時に、`type(x).__call__(x, arg1, ...)` のように暗黙的に変換される。

以下は、すべて呼び出し可能なオブジェクトである。

  * 組み込み関数
  * 組み込みオブジェクトのメソッド
  * ユーザ定義関数
  * クラスオブジェクト
  * クラスインスタンスのメソッド

これらは、次のような属性を持つ。

| 特殊属性 | 意味 |
|:--|:----|
| `__name__` | 関数やメソッドの名前 |
| `__dict__` | 属性辞書 |
| `__doc__` | ドキュメンテーション文字列。なければ `None` |

ユーザ定義関数オブジェクトは、関数定義の実行時に作成される。関数定義は実行可能な文であり、関数定義を実行すると、現在のローカルな名前空間内で関数名を関数オブジェクトに束縛する（関数本体を実行しない。関数本体は関数が呼び出された時にのみ実行される）。

また、ユーザ定義関数オブジェクトは、ラムダ式の評価により返される。ラムダ式で得られる関数オブジェクトは、関数名の束縛を伴わないことから、**無名関数**（anonymous function）と呼ばれる。ラムダ式の構文は次のとおり。

``` python
lambda [引数リスト]: 式
```

引数リストは省略できる。無名関数は関数本体の `式` の評価結果を返す。無名関数を変数に代入することもできるが、これは PEP 8 で非推奨とされる。

In [ ]:
f = lambda x, y: x + y  # PEP 8 非推奨
assert f(2, 3) == type(f).__call__(f, 2, 3)

任意のクラスに `__call__()` メソッドを定義すれば、そのクラスから生成されるインスタンスは呼び出し可能となる。


In [ ]:
class Chatter:
    def __call__(self):
        print("How are you doing?")

c = Chatter()
c()

How are you doing?


デコレーター
------------

Python において、関数はオブジェクトであるから、変数に代入したり、他の関数の引数として渡すことができる。他の関数の引数として渡される関数を**コールバック関数**（callback function）と呼び、反対に、関数を引数に取る関数を**高階関数**（higher-order function）と呼ぶ。

Python では、ネストして定義した関数を戻り値として返すこともできる。関数を返す関数を**ファクトリー関数**と呼ぶ。とくに、関数を返す高階関数を、Python では**関数デコレーター**または簡単に**デコレーター**（decorator）と呼ぶ。ある関数にデコレーターを適用することを「デコレートする」と表現する。

デコレーターを使うことにより、既存の関数 `my_func()` そのものの中身を変えず、関数の前後に処理を追加して実行するということができる：

In [ ]:
def decorator(func):
    def wrap_function():
        """wrapper function"""
        print("some kind of pre-processing")
        ret = func()
        print("some post-processing")
        return ret

    return wrap_function

def my_func():
    """original function"""
    print("original function")

my_func = decorator(my_func)
my_func()

some kind of pre-processing
original function
some post-processing


この `decorator()` 関数はデコレーターであり、内部関数として `wrap_function()` を定義し返している。`decorator()` の引数 `func` は関数であり、これを `wrap_function()` の中で呼び出すことで、`wrap_function()` は `func()` 関数の実行前後に処理を追加するものとなっている。`wrap_function()` のような関数は **wrapper 関数**と呼ばれ、`func`（引数として渡される関数）は **wrapped 関数**と呼ばれる。

最後の 2 行で変数 `my_func` に `decorator()` 関数の戻り値（内部関数である `wrap_function()`）を代入し、 `my_func()` を呼び出している。この呼び出しは、実際には `wrap_function()` を実行することになる。

ただし、この書き方では、もとの関数 `my_func()` の関数定義と、変数 `my_func` への関数オブジェクトの代入が離れていて、少々具合がよくない。というのは、その間に `my_func()` の呼び出しを書いてしまうと、もとの関数 `my_func()` が実行できてしまうからである。そこで、Python では上記のコードと等価な別の書き方をサポートしている：

In [ ]:
def decorator(func):
    def wrap_function():
        """wrapper function"""
        print("some kind of pre-processing")
        ret = func()
        print("some post-processing")
        return ret

    return wrap_function

@decorator
def my_func():
    """original function"""
    print("original function")

my_func()

some kind of pre-processing
original function
some post-processing


このように、関数定義の直前に `@decorator` と書くと、Python のインタープリターは、暗黙のうちに関数定義の直後に

``` python
my_func = decorator(my_func)
```

を自動で挿入してくれる。これにより、もとの関数 `my_func()` そのものを実行できる余地がなくなる。

代入の効果を確認するために、特殊属性 `__name__` と `__doc__` を調べてみると

In [ ]:
assert my_func.__name__ == "wrap_function"
assert my_func.__doc__ == "wrapper function"

となっている。いずれも、`decorator()` 関数が返した関数 `wrap_function()` のものになっている。代入したのであるから当たり前の結果なのであるが、実はこの結果は行き過ぎでもある。これは後の課題とする。

複数のデコレーターを適用することもできる：

``` python
@decorator1
@decorator2
def func():
    pass
```

このコードは次の代入文と等価である：

``` python
func = decorator1(decorator2(func))
```

このように、デコレーターは内側の `decorator2` から外側に向かって順に適用される。

wrapper 関数の汎用化
--------------------

デコレーターを定義するとき、内部の wrapper 関数を、ある 1 つの wrapped 関数に依存した書き方（つまり、その関数の引数や戻り値を決め打ちした書き方）をする場合、せっかく定義したデコレーターが他の関数に適用できなくなる。あるいは、将来 wrapped 関数を変更すると、デコレーターの変更も必要となる。そこで、wrapper 関数は、できるだけ wrapped 関数の引数や戻り値を決め打ちして実装することがないようにすべきである。

一方で、wrapper 関数の引数は、wrapped 関数の引数を受け付けるものでなければならない。wrapper 関数の外では wrapped 関数の呼び出しを wrapper 関数の呼び出しに置き換え、しかも、wrapper 関数の中では wrapped 関数を呼び出すからである。前出の wrapper 関数 `wrap_function()` は、wrapped 関数が引数を取らないと決め打ちしていたので、引数を取る wrapped 関数に対しては、TypeError 例外が発生してしまう。

In [ ]:
@decorator
def my_func_with_arg(arg):
    """original function with argument"""
    print(f"original function with argument {arg}")

try:
    my_func_with_arg(1)
except TypeError as err:
    print(f"{type(err).__name__}: {err}")

TypeError: decorator.<locals>.wrap_function() takes 0 positional arguments but 1 was given


wrapped 関数の引数がどのようなものであっても、それを受け付けることができる wrapper 関数を定義するには、可変長引数を使うとよい。wrapper 関数の引数を `*args, **kwargs` と定義し、中で呼び出す wrapped 関数には アンパック `*args, **kwargs` を渡すようにすれば、すべての wrapped 関数の引数に対応できる：

In [ ]:
def decorator(func):
    def wrap_function(*args, **kwargs):
        """wrapper function"""
        print("some kind of pre-processing")
        ret = func(*args, **kwargs)
        print("some post-processing")
        return ret

    return wrap_function

@decorator
def my_func():
    """original function"""
    print("original function")

@decorator
def my_func_with_arg(arg):
    """original function with argument"""
    print(f"original function with argument {arg}")

my_func()
my_func_with_arg(1)

some kind of pre-processing
original function
some post-processing
some kind of pre-processing
original function with argument 1
some post-processing


クロージャー
------------

デコレーターを定義するとき、スコープが気になるかもしれない。 wrapper 関数を基点として考えると、デコレーターはエンクロージングスコープである。つまり、wrapper 関数の仮引数でもなく、wrapper 関数自身のローカル変数でもない変数（こういう変数は**自由変数**と呼ばれる）は、デコレーターの名前空間が探索される。

ここまではよいが、wrapper 関数が呼び出されるのは、デコレーターの外、つまり、デコレーターから wrapper 関数が返されてデコレーターの処理が終了した後である点が問題となる。というのは、関数の内部で変数が束縛されたオブジェクトの生存期間は、関数が処理されている間だけであるからである。呼び出された関数とその中で使用されるオブジェクトは、**スタック領域**と呼ばれるメモリ上の領域に格納され、関数の処理が終了するとスタック領域から取り除かれる。したがって、wrapper 関数が呼び出された時点では、デコレーターの変数が束縛されたオブジェクトは消えており、その変数の値を参照できないことになってしまう。

Python は、この問題を解決するため、エンクロージングスコープだけ特別な扱いをしている。実は、エンクロージングスコープは内側の関数の状態として閉じ込められた形で存在している。ユーザー定義関数は、次の特殊属性に、自身の定義が実行された時の自由変数に関する情報を保持している。

  * `function.__code__.co_freevars`: タプルで、自由変数の名前が格納される。
  * `function.__closure__`: タプルで、各要素の `cell_contents` 属性に自由変数の値が格納される。

閉じ込められた自由変数は以降、内側の関数からしか参照できないため名前空間を汚さない。この仕組みを**クロージャー**（closure）または**関数閉包**と呼ぶ。クロージャーによって、外側の関数の処理が終了した後に内側の関数を呼び出すことになっても、外側の関数のローカル変数を参照するということが可能になっているわけである。

実際、次のコードは問題なく実行できる:

In [ ]:
def outer():
    x = 1

    def inner():
        print(x)

    return inner

foo = outer()
foo()
print("----")
for n, c in zip(foo.__code__.co_freevars, foo.__closure__):
    print("{}: {}".format(n, c.cell_contents))

1
----
x: 1


外側の関数の変数を上書きする場合は、nonlocal 宣言が必要であったが、外側の関数の名前空間は関数が実行されるたびに新しく生成されることに注意する。たとえば、次のコードでは、外側の関数の変数を内側の関数がインクリメントするが、外側の関数を実行するたびにその変数は初期化される:

In [ ]:
def counter_factory():
    c = 0

    def counter():
        nonlocal c  # １つ外側の変数 c を参照する
        c += 1
        return c

    return counter

counter1 = counter_factory()
print("counter1: {}".format(counter1()))
print("counter1: {}".format(counter1()))
print("counter1: {}".format(counter1()))
counter2 = counter_factory()
print("counter2: {}".format(counter2()))
print("counter2: {}".format(counter2()))
print("counter2: {}".format(counter2()))

counter1: 1
counter1: 2
counter1: 3
counter2: 1
counter2: 2
counter2: 3


引数を受け取るデコレーター
--------------------------

クロージャーによって、デコレーターのローカル変数は wrapper 関数から参照できることがわかった。そうなると、デコレーターに引数を渡したくなる。

まず、&commat; 構文で引数を渡せるのかというと、これは問題ない。&commat; 構文には、式であれば、なんでも書ける（セイウチ演算子を含む代入式も可）。

``` python
@<式>
```

&commat; 構文に、デコレーターを返す関数の呼び出しを書くと、&commat; 構文の中で引数を渡すことができる。デコレーターを返す関数（紛らわしいがこれもデコレーターと呼ばれる）は、内部関数としてデコレーターを定義し、その内部関数として wrapper 関数を定義する。

In [ ]:
def decorator(option):
    def inner_decorator(func):
        def wrap_function(*args, **kwargs):
            """wrapper function"""
            print(f"some kind of pre-processing with {option=}")  # option の値を利用できる
            ret = func(*args, **kwargs)
            print(f"some post-processing with {option=}")  # option の値を利用できる
            return ret

        return wrap_function

    return inner_decorator

@decorator("foo")
def my_func(arg):
    """original function"""
    print(f"original function: {arg=}")

my_func(1)
assert my_func.__name__ == "wrap_function"
assert my_func.__doc__ == "wrapper function"

some kind of pre-processing with option='foo'
original function: arg=1
some post-processing with option='foo'


`decorator()` の仮引数 `option` は、 `wrap_function()` 関数にとっての自由変数であり、 `inner_decorator()` 関数にとっての自由変数でもある。ここで 2 段階のクロージャーによって、`option` は `wrap_function()` 関数の呼び出しにおいて参照可能である。

`my_func(arg)` の呼び出しは、`decorator(option)(my_func)(arg)` の呼び出しと等価である。

呼び出し可能型を使って次のように書くこともできる。

In [ ]:
class Decorator:
    def __call__(self, option):
        self.option = option
        return self.inner_decorator

    def inner_decorator(self, func):
        def wrap_function(*args, **kwargs):
            """wrapper function"""
            print(f"some kind of pre-processing with {self.option=}")  # self.option の値を利用できる
            ret = func(*args, **kwargs)
            print(f"some post-processing with {self.option=}")  # elf.option の値を利用できる
            return ret
        return wrap_function

decorator = Decorator()

@decorator("foo")
def my_func(arg):
    """original function"""
    print(f"original function: {arg=}")

my_func(1)
assert my_func.__name__ == "wrap_function"
assert my_func.__doc__ == "wrapper function"

some kind of pre-processing with self.option='foo'
original function: arg=1
some post-processing with self.option='foo'


functools.wraps
---------------

デコレーターを適用すると、どのオリジナル関数も、`__name__` 属性と `__doc__` 属性が wrapper 関数のものに置き換えられてしまう。`help()` 関数やエディタの入力支援はこれらの属性を表示するので、混乱の原因となる。置き換えられないようにするには、標準ライブラリの `functools` モジュールが提供する `functools.wraps()` 関数を wrapper 関数に適用すればよい。`functools.wraps()` 関数は wrapped 関数を引数に取るので、呼び出し形式のデコレーターとして、次のように wrapper 関数に適用する。

In [ ]:
from functools import wraps

def decorator(option):
    def inner_decorator(func):
        @wraps(func)
        def wrap_function(*args, **kwargs):
            """wrapper function"""
            print(f"some kind of pre-processing with {option=}")  # option の値を利用できる
            ret = func(*args, **kwargs)
            print(f"some post-processing with {option=}")  # option の値を利用できる
            return ret

        return wrap_function

    return inner_decorator

@decorator("foo")
def my_func(arg):
    """original function"""
    print(f"original function: {arg=}")

my_func(1)
assert my_func.__name__ == "my_func"
assert my_func.__doc__ == "original function"

some kind of pre-processing with option='foo'
original function: arg=1
some post-processing with option='foo'


なお、`functools.wraps()` 関数は、 wrapper 関数の属性を wrapped 関数の属性で更新する `functools.update_wrapper()` 関数をデコレーターとして使用できるようにしたものである。`functools.wraps(wrapped)(wrapper)` は、`functools.update_wrapper(wrapper, wrapped)` と等価である。

メモ化
------

`functools` モジュールは、便利なデコレーターを提供している。

``` python
functools.cache(user_function)
```

これは、関数 `user_function` が[メモ化](https://ja.wikipedia.org/wiki/メモ化)された関数を返すデコレーターである。Python 3.9 で追加された。Python 3.8 以前なら、`functools.lru_cache(maxsize=None)` が同等の関数を返す。

メモ化された関数は、引数と戻り値をキャッシュに記憶しておき、2 回目以降の呼び出しに同じ引数を見つけたときは記憶してある値を返す。これにより、同じ計算を繰り返すことがなくなって効率が良くなる。

ただし、キャッシュの性質上、以下の点に注意する。

  * キャッシュには、引数をキー、戻り値をキーの値とする辞書が使われるので、**引数はハッシュ可能な値でなければならない**。キーワード引数の順序が異なる場合は、異なるキャッシュとして保存されるため、メモ化の効果がない。
  * メモ化される関数は**参照透過性**（同じ引数に対して同じ戻り値を返すこと）を持つ必要がある。参照透過性を持たない関数のメモ化では、その関数の本来の結果を受け取ることができなくなってしまう。
  * キャッシュはプログラムが終了すると失われる。

In [ ]:
from functools import cache

@cache
def func(num):
    print(f"{num=}: 関数の中に入って処理しています")
    return num + 1

r1 = func(100)
r2 = func(200)
r3 = func(100)  # キャッシュから値を返すので、実際には関数本体が実行されない
assert [r1, r2, r3] == [101, 201, 101]  # r3 はキャッシュからの値

num=100: 関数の中に入って処理しています
num=200: 関数の中に入って処理しています


メモ化は、関数に同じ引数が何度も渡されるようなアルゴリズムで使用すると大きな効果を発揮する。そのようなアルゴリズムの例が[動的計画法](https://ja.wikipedia.org/wiki/動的計画法)による再帰的アルゴリズムである。以下のコードでは、`fibonacci()` は[フィボナッチ数](https://ja.wikipedia.org/wiki/フィボナッチ数)を返す再帰関数であり（初項は 1 とする）、メモ化を使用しない場合の実行時間を計測している。

In [ ]:
def fibonacci(x):
  if x==1 or x==2:
    return 1
  else:
    return fibonacci(x-1) + fibonacci(x-2)

%timeit -n 30 -r 3 fibonacci(20)

10.4 ms ± 1.34 ms per loop (mean ± std. dev. of 3 runs, 30 loops each)


この `fibonacci()` 関数をメモ化して実行時間を計測すると、1/10000 近くに実行時間が短縮化されることがわかる:

In [ ]:
from functools import cache

@cache
def fibonacci(x):
  if x==1 or x==2:
    return 1
  else:
    return fibonacci(x-1) + fibonacci(x-2)

%timeit -n 30 -r 3 fibonacci(20)

The slowest run took 11.41 times longer than the fastest. This could mean that an intermediate result is being cached.
521 ns ± 557 ns per loop (mean ± std. dev. of 3 runs, 30 loops each)


クラスデコレーター
------------------

クラスオブジェクトを引数にとり、クラスオブジェクトを返す関数を**クラスデコレーター**という。

In [ ]:
def class_decorator(cls):
    cls.x = "sample"
    return cls

class MyClass:
    """original class"""
    pass

MyClass = class_decorator(MyClass)

# インスタンスを生成
m = MyClass()
print(f"{m.x=}")

m.x='sample'


関数デコレーターと同様に、&commat; 構文が使える。

In [ ]:
def class_decorator(cls):
    cls.x = "sample"
    return cls

@class_decorator
class MyClass:
    """original class"""
    pass

# インスタンスを生成
m = MyClass()
print(f"{m.x=}")

m.x='sample'


クラスデコレーターでできることは、メタクラスで `__init__()` をオーバーライドする方法でも実現できる。

複数のデコレーターを適用することもできる:

``` python
@f1(arg)
@f2
class Foo: pass
```

は、だいたい次と等価である。

``` python
class Foo: pass
Foo = f1(arg)(f2(Foo))
```